# Highlights 
## Used Pandas profiling for the first time and just amazed with it's utility, used np.log1p()[Explained in detail below], also explored fancyimpute

## # Suggestions and views are most welcomed

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/train.csv")
test = pd.read_csv("/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/test.csv")

In [ ]:
train

**By problem statement I observed that score is calculated using MEAN-SQUARE-LOG-ERROR. It is so because the target has very large standard deviation(std)(shown below). So for prediction purpose I wanted to reduce std. This is done by taking[numpy.log1p] of the target variable. **

In [ ]:
def log1p(vec):
    return np.log1p(abs(vec))

def expm1(x):
    return np.expm1(x)

In [ ]:
train_Y = train['Cost']

In [ ]:
train_Y

In [ ]:
dev1 = np.std(train_Y)
dev1

In [ ]:
train_Y = log1p(train_Y)
train_X = train.drop(['Cost'], axis=1)
test_X = test
train_X

In [ ]:
train_Y

In [ ]:
dev2 = np.std(train_Y)
dev2

# Using Pandas Profiling for EDA 

In [ ]:
import pandas_profiling 

In [ ]:
train.profile_report()

In [ ]:
train_X = train_X.drop(['Customer Id', 'Artist Name', 'Artist Reputation', 'Transport'], axis = 1)
test_X = test.drop(['Customer Id', 'Artist Name', 'Artist Reputation', 'Transport'], axis = 1)

In [ ]:
train_X['State'] = train_X['Customer Location'].map(lambda x:x.split()[-2])
train_X.drop(['Customer Location'], inplace = True, axis = 1)
test_X['State'] = test_X['Customer Location'].map(lambda x:x.split()[-2])
test_X.drop(['Customer Location'], inplace = True, axis = 1)

In [ ]:
train_X['Scheduled Date'] = pd.to_datetime(train_X['Scheduled Date'])
train_X['Delivery Date'] = pd.to_datetime(train_X['Delivery Date'])
train_X['scheduleDiff'] = (train_X['Scheduled Date'] - train_X['Delivery Date']).map(lambda x:str(x).split()[0])
train_X['scheduleDiff'] = pd.to_numeric(train_X['scheduleDiff'])
test_X['Scheduled Date'] = pd.to_datetime(test_X['Scheduled Date'])
test_X['Delivery Date'] = pd.to_datetime(test_X['Delivery Date'])
test_X['scheduleDiff'] = (test_X['Scheduled Date'] - test_X['Delivery Date']).map(lambda x:str(x).split()[0])
test_X['scheduleDiff'] = pd.to_numeric(test_X['scheduleDiff'])

In [ ]:
train_X['dday'] = train_X['Delivery Date'].dt.day
train_X['dmonth'] = train_X['Delivery Date'].dt.month
train_X['dyear'] = train_X['Delivery Date'].dt.year
train_X['ddayofweek'] = train_X['Delivery Date'].dt.dayofweek

test_X['dday'] = test_X['Delivery Date'].dt.day
test_X['dmonth'] = test_X['Delivery Date'].dt.month
test_X['dyear'] = test_X['Delivery Date'].dt.year
test_X['ddayofweek'] = test_X['Delivery Date'].dt.dayofweek

test_X.head()

In [ ]:
train_X.drop(['Delivery Date', 'Scheduled Date'], inplace=True, axis=1)
test_X.drop(['Delivery Date', 'Scheduled Date'], inplace=True, axis=1)
train_X.head()

**Dealing with categorical columns having missing values**

In [ ]:
#1. Function to replace NAN values with mode value
def impute_nan_most_frequent_category(DataFrame,ColName):
    # .mode()[0] - gives first category name
     most_frequent_category=DataFrame[ColName].mode()[0]
    
    # replace nan values with most occured category
     DataFrame[ColName + "_Imputed"] = DataFrame[ColName]
     DataFrame[ColName + "_Imputed"].fillna(most_frequent_category,inplace=True)
#2. Call function to impute most occured category
for Columns in ['Material', 'Remote Location']:
    impute_nan_most_frequent_category(train_X,Columns)
    impute_nan_most_frequent_category(test_X,Columns)
    
# Display imputed result
train_X[['Material','Material_Imputed','Remote Location','Remote Location_Imputed']].head(10)
#3. Drop actual columns
train_X = train_X.drop(['Material', 'Remote Location'], axis = 1)
test_X = test_X.drop(['Material', 'Remote Location'], axis = 1)

In [ ]:
#test_X.profile_report()

In [ ]:
train_X

In [ ]:
# Get list of categorical variables
s = (train_X.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

**One-Hot Encoding Categorical Data**

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

onehot_X_train = one_hot_encoder.fit_transform(train_X[object_cols])
onehot_X_test = one_hot_encoder.fit_transform(test_X[object_cols])
onehot_X_train = pd.DataFrame(onehot_X_train)
onehot_X_test = pd.DataFrame(onehot_X_test)

num_X_train = train_X.drop(object_cols, axis=1)
num_X_valid = test_X.drop(object_cols, axis=1)

onehot_X_train = pd.concat([num_X_train, onehot_X_train], axis=1)
onehot_X_test = pd.concat([num_X_valid, onehot_X_test], axis=1)


In [ ]:
onehot_X_train

**Imputing missing values using fancyimpute**

In [ ]:
from fancyimpute import KNN 
from fancyimpute import IterativeImputer
#imputer = KNN(k=5)
imputer = IterativeImputer()
#train_ = train[columns]
fullset = pd.concat([onehot_X_train, onehot_X_test]) 
#fancy impute removes column names
fullset = pd.DataFrame(imputer.fit_transform(fullset))
#fullset.columns = columns
fullset

In [ ]:
train_df = fullset[:onehot_X_train.shape[0]]
test_df = fullset[onehot_X_train.shape[0]:]

**Data Normalisation**

In [ ]:
import sklearn
scaler = sklearn.preprocessing.StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(train_df), columns=train_df.columns)
X_test = pd.DataFrame(scaler.transform(test_df), columns=test_df.columns)


**RandomForest Regressor**

Earlier I tried this model and procedure

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, train_Y)

In [ ]:
y_prediction_rf = rf.predict(X_test)

In [ ]:
y_prediction_rf

**Tuning RF**

In [ ]:
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_grid = [
{'n_estimators': [50,100,250,500], 
 'max_depth': [10, 50, 100], 'bootstrap': [True, False]}
]


In [ ]:
random_search_forest = RandomizedSearchCV(rf, param_grid, cv=10)
random_search_forest.fit(X_train, train_Y)

In [ ]:
tuned_rf_best_random = random_search_forest.best_estimator_

In [ ]:
tuned_rf_best_random

In [ ]:
tuned_rf_best_random.fit(X_train, train_Y)

In [ ]:
tuned_rf_random_pred = tuned_rf_best_random.predict(X_test)

In [ ]:
tuned_rf_random_pred

**LGBM**

Nextly used this model

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
lgbm = LGBMRegressor()
lgbm.fit(X_train, train_Y)
y_test_pred_lgbm = lgbm.predict(X_test)

In [ ]:
y_test_pred_lgbm

In [ ]:
y_test_pred_lgbm2 = expm1(y_test_pred_lgbm)

In [ ]:
y_test_pred_lgbm2

In [ ]:
df = pd.DataFrame()

In [ ]:
df['Customer Id'] = test['Customer Id']
df['Cost'] = y_test_pred_lgbm2
df['Cost'] = df['Cost'].abs()

In [ ]:
df

In [ ]:
df.to_csv('prediction1.csv', index = False)